This notebook provides functions to compute the independent weight of a given binary source (P) over d random variables, and the success parameters of the independent source(s) that achieve that weight. For d>6, the algorithm may be too slow. This algorithm does not require P>0.

In [12]:
import numpy as np
import itertools

In [13]:
def createIndependentSource(params):
    '''Given a list (params) of [0,1]-valued Bernoulli parameters, returns
    a list (P) of length 2^d, the probabilities of the outcomes (in lexicographical order)
    of independent Bernoulli random variables'''
    d = len(params)
    P = []
    Range = list(map(lambda x: ''.join(x),itertools.product(['0', '1'], repeat=d))) 
    for k in range(len(Range)):
        pk = 1.
        for j in range(d):
            pk *= (params[j]**float(Range[k][j]))*(1.-params[j])**(1.-float(Range[k][j]))
        P.append(pk)
    return P

In [14]:
def is_invertible(a):
    '''Given a square numpy array (a), returns True if a is invertible and False if not'''
    return a.shape[0] == a.shape[1] and np.linalg.matrix_rank(a) == a.shape[0]

In [15]:
def compatible(c,P):
    '''Given a list (c) of length d, the configuration, returns True if the configuration creates
    0-probability outcomes x whenever P(x)=0, and False if P(x)=0 for some x which does not have
    0-probability under c'''
    d = int(np.log2(len(P)))
    Range = list(map(lambda x: ''.join(x),itertools.product(['0', '1'], repeat=d)))
    Q = createIndependentSource(c)
    for j in range(len(P)):
        if P[j]==0 and Q[j]!=0:return False
    return True

In [16]:
def computeVerticesHoles(P,k,c,epsilon=10**-5):
    '''Takes a list (P) of length 2^d (a probability distribution), a string (k) of length d (a binary outcome),
    a list (c) of length d (a configuration -- which coordinates are deterministic), and a tolerance (epsilon).
    Returns the maximum of the objective function in the polyhedron Q_k, and the d Bernoulli parameters
    corresponding to the maximiser on the polyhedron'''
    Q = createIndependentSource(c)
    d = len(k)
    Range = list(map(lambda x: ''.join(x),itertools.product(['0', '1'], repeat=d)))
    Qholes = [x for x in range(2**d) if Q[x]==0]
    variables = [x for x in range(d) if 0<c[x]<1]
    e = len(variables)
    A = []
    b = []
    K = Range.index(k)
    for j in range(len(Range)):
        if j!=K and P[j]>0 and Q[j]>0:
            b.append(np.log(P[j]/P[K]))
            A.append([])
            for i in variables:
                if Range[j][i]!=Range[K][i]: A[-1].append(1)
                else:A[-1].append(0)
    
    subsets = list(itertools.combinations(list(range(np.shape(A)[0])),e)) #All choose(2^d-1,d) square subsystems
    A = np.array(A)
    b = np.array(b)
    vertices = []
    for subset in subsets:
        Atilde = A[list(subset),:]
        try:
            ytilde = np.linalg.solve(Atilde,b[list(subset)])
            if np.all(np.dot(A,ytilde)<=b+epsilon):#epsilon added when checking feasibility of potential vertex
                                                    #Numerically unstable otherwise, may fail to find a true vertex
                vertices.append(ytilde)
        except np.linalg.LinAlgError:
            pass
        
    critVals = []
    for j in range(len(vertices)):
        critVals.append(P[K]*np.prod(1+np.exp(vertices[j])))
    M = max(critVals)
    yhat = vertices[critVals.index(M)]
    q = []
    l = 0

    for j in range(d):
        if j in variables:
            q.append(np.exp(yhat[l])**(1-2*float(k[l]))/\
                     (1+np.exp(yhat[l])**(1-2*float(k[l]))))
            l+=1
        else:
            q.append(c[j])
        
    
    return M,q

In [19]:
def indWeightHoles(P):
    '''Given a list (P) of length 2^d, representing a probability distribution over d binary random variables,
    with outcomes in lexicographical order, returns the independent weight and the Bernoulli parameters
    of the component(s) where the independent weight is achieved'''
    d = int(np.log2(len(P)))
    C = list(itertools.product([0,1,0.5],repeat=d))
    compatibles = []
    for c in C:
        if compatible(c,P) and c.count(0.5)>0:
            compatibles.append(c)

    Range = list(map(lambda x: ''.join(x),itertools.product(['0', '1'], repeat=d)))
    
    
    objectives = []
    maximisers = []

    for c in compatibles:
        Q = createIndependentSource(c)
        for I in Range:
            if Q[Range.index(I)]>0:
                Obj,Max = computeVerticesHoles(P,I,c)
                objectives.append(Obj)
                maximisers.append(Max)
      
            
    Lambda = max(objectives)
    return Lambda, maximisers[objectives.index(Lambda)] 



In [20]:
###Example###

P = [0.25,0.25,0.25,0.25]#Uniform distribution on {0,1}^2
indWeightHoles(P)

(1.0, [0.5, 0.5])